In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime as dt
import pandas as pd
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import pyspark
from pyspark.sql import SparkSession

In [3]:
from pyspark.sql.session import SparkSession

spark = SparkSession.builder.getOrCreate()

In [4]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

In [5]:
import requests
import json
import datetime as dt
from hdfs import InsecureClient
client = InsecureClient('http://localhost:9870', user='big')

In [6]:
JDBC = {
    'url':'jdbc:oracle:thin:@decorona1_high?TNS_ADMIN=/home/big/project/db/Wallet_DECORONA1'
    ,'props':{
        'user':'dw_jjy'
       ,'password':'123qwe!@#QWE'
    }   
}

In [7]:
path = '/news_data/article'
co_patient_json = spark.read.json(path, encoding='UTF-8')
co_patient_json.show()

+-------------------------------+------------------------------+
|                           data|                          meta|
+-------------------------------+------------------------------+
|   [{"상장사, 4분기 더 강한 ...|{{뉴스제목}, 네이버 실시간 ...|
|[{코스피, 영란은행이 만든 반...|{{뉴스제목}, 네이버 실시간 ...|
+-------------------------------+------------------------------+



In [8]:
file_name = '/news_data/article/realtimenews_' + cal_std_day(0) + '.json'
tmp = spark.read.json(file_name, encoding='UTF-8')
tmp.show(10)

+----------------------------+------------------------------+
|                        data|                          meta|
+----------------------------+------------------------------+
|[{"상장사, 4분기 더 강한 ...|{{뉴스제목}, 네이버 실시간 ...|
+----------------------------+------------------------------+



In [9]:
data=[]
for r1 in tmp.select('data').toLocalIterator():
    if not r1.data:
        continue
    for r2 in r1.data:
        data.append(r2)
data

[Row(TITLE='"상장사, 4분기 더 강한 충격이 온다"'),
 Row(TITLE="미래에셋증권, 미 주식 거래 하루 '21시간'으로 연장"),
 Row(TITLE='롤러코스터 증시… “코스피 1900대까지 하락할 수도”'),
 Row(TITLE='한투 "SK하이닉스, 반도체 악재 선반영…목표주가 유지"'),
 Row(TITLE='"세계국채지수(WGBI) 편입되면 금리·환율 안정 기대"'),
 Row(TITLE='“두산, 경쟁사 대비 로봇사업 가치 현저히 저평가” 목표가 18만 원'),
 Row(TITLE="올리패스, 나스닥 상장 제약사 美 '반다'와 공동 R&D 계약"),
 Row(TITLE='NH투자증권·웨이버스 등 42개사 의무보유 해제…1억7296만주'),
 Row(TITLE='"실적은 좋은데" WCP 상장 첫날 폭락..반등 가능성은?'),
 Row(TITLE='비패밀리 메신저, 10월 출석체크 오픈 이벤트 실시'),
 Row(TITLE='유니온커뮤니티, 55억 규모 육군 출입통제 구축사업 계약'),
 Row(TITLE='애드바이오텍, 6회차 CB 전환가액 하향 조정'),
 Row(TITLE='현대퓨처넷, 한화건설과 185억 규모 공급계약 체결'),
 Row(TITLE='안다자산운용 "SK케미칼 자사주 추가 소각해야"'),
 Row(TITLE='한창, 진도 자원순환센터 개소 기념식·기술설명회 개최'),
 Row(TITLE='미래에셋 글로벌엑스 재팬, ETF 3종목 동경거래소 신규 상장'),
 Row(TITLE='박민영과 열애설 난 빗썸 회장?… "회장 직함 가진 사람 없어"'),
 Row(TITLE='미래에셋증권, 미국주식 거래 아침 9시부터 가능해진다'),
 Row(TITLE="[특징주] 체면 구긴 '대어' WCP에 투자자 울상…공모가比 26%↓"),
 Row(TITLE='파인텍, 자사주 25만9819주 소각 결정'),
 Row(TITLE='에이치엘사이언스, 30억 규모 자사주 신탁계약 체결'),
 Row(TITLE='씨티그룹 "연준 인식 바뀔 

In [10]:
data2=spark.createDataFrame(data)
data2.show()

+---------------------------------+
|                            TITLE|
+---------------------------------+
|   "상장사, 4분기 더 강한 충격...|
| 미래에셋증권, 미 주식 거래 하...|
|   롤러코스터 증시… “코스피 19...|
|   한투 "SK하이닉스, 반도체 악...|
|    "세계국채지수(WGBI) 편입되...|
|  “두산, 경쟁사 대비 로봇사업 ...|
| 올리패스, 나스닥 상장 제약사 ...|
|   NH투자증권·웨이버스 등 42개...|
|     "실적은 좋은데" WCP 상장 ...|
|  비패밀리 메신저, 10월 출석체...|
|  유니온커뮤니티, 55억 규모 육...|
|   애드바이오텍, 6회차 CB 전환...|
|  현대퓨처넷, 한화건설과 185억...|
| 안다자산운용 "SK케미칼 자사주...|
| 한창, 진도 자원순환센터 개소 ...|
|  미래에셋 글로벌엑스 재팬, ET...|
| 박민영과 열애설 난 빗썸 회장?...|
|미래에셋증권, 미국주식 거래 아...|
|    [특징주] 체면 구긴 '대어' ...|
|     파인텍, 자사주 25만9819주...|
+---------------------------------+
only showing top 20 rows



In [15]:
news_day =data2.withColumn('STD_DAY',current_date())
news_day = news_day[[col('STD_DAY'),col('TITLE')]]
news_day.show()

+----------+---------------------------------+
|   STD_DAY|                            TITLE|
+----------+---------------------------------+
|2022-09-30|   "상장사, 4분기 더 강한 충격...|
|2022-09-30| 미래에셋증권, 미 주식 거래 하...|
|2022-09-30|   롤러코스터 증시… “코스피 19...|
|2022-09-30|   한투 "SK하이닉스, 반도체 악...|
|2022-09-30|    "세계국채지수(WGBI) 편입되...|
|2022-09-30|  “두산, 경쟁사 대비 로봇사업 ...|
|2022-09-30| 올리패스, 나스닥 상장 제약사 ...|
|2022-09-30|   NH투자증권·웨이버스 등 42개...|
|2022-09-30|     "실적은 좋은데" WCP 상장 ...|
|2022-09-30|  비패밀리 메신저, 10월 출석체...|
|2022-09-30|  유니온커뮤니티, 55억 규모 육...|
|2022-09-30|   애드바이오텍, 6회차 CB 전환...|
|2022-09-30|  현대퓨처넷, 한화건설과 185억...|
|2022-09-30| 안다자산운용 "SK케미칼 자사주...|
|2022-09-30| 한창, 진도 자원순환센터 개소 ...|
|2022-09-30|  미래에셋 글로벌엑스 재팬, ET...|
|2022-09-30| 박민영과 열애설 난 빗썸 회장?...|
|2022-09-30|미래에셋증권, 미국주식 거래 아...|
|2022-09-30|    [특징주] 체면 구긴 '대어' ...|
|2022-09-30|     파인텍, 자사주 25만9819주...|
+----------+---------------------------------+
only showing top 20 rows



In [16]:
realtime_news = news_day.select(
    news_day.STD_DAY.alias('STD_DAY')
    ,news_day.TITLE.alias('TITLE')
)
realtime_news.printSchema()

root
 |-- STD_DAY: date (nullable = false)
 |-- TITLE: string (nullable = true)



In [17]:
realtime_news.show()

+----------+---------------------------------+
|   STD_DAY|                            TITLE|
+----------+---------------------------------+
|2022-09-30|   "상장사, 4분기 더 강한 충격...|
|2022-09-30| 미래에셋증권, 미 주식 거래 하...|
|2022-09-30|   롤러코스터 증시… “코스피 19...|
|2022-09-30|   한투 "SK하이닉스, 반도체 악...|
|2022-09-30|    "세계국채지수(WGBI) 편입되...|
|2022-09-30|  “두산, 경쟁사 대비 로봇사업 ...|
|2022-09-30| 올리패스, 나스닥 상장 제약사 ...|
|2022-09-30|   NH투자증권·웨이버스 등 42개...|
|2022-09-30|     "실적은 좋은데" WCP 상장 ...|
|2022-09-30|  비패밀리 메신저, 10월 출석체...|
|2022-09-30|  유니온커뮤니티, 55억 규모 육...|
|2022-09-30|   애드바이오텍, 6회차 CB 전환...|
|2022-09-30|  현대퓨처넷, 한화건설과 185억...|
|2022-09-30| 안다자산운용 "SK케미칼 자사주...|
|2022-09-30| 한창, 진도 자원순환센터 개소 ...|
|2022-09-30|  미래에셋 글로벌엑스 재팬, ET...|
|2022-09-30| 박민영과 열애설 난 빗썸 회장?...|
|2022-09-30|미래에셋증권, 미국주식 거래 아...|
|2022-09-30|    [특징주] 체면 구긴 '대어' ...|
|2022-09-30|     파인텍, 자사주 25만9819주...|
+----------+---------------------------------+
only showing top 20 rows



In [18]:
type(realtime_news)

pyspark.sql.dataframe.DataFrame

In [19]:
#__stack_dataframe

pd_news=realtime_news.to_pandas_on_spark()

pd_news

,STD_DAY,TITLE
0,2022-09-30,"""상장사, 4분기 더 강한 충격이 온다"""
1,2022-09-30,"미래에셋증권, 미 주식 거래 하루 '21시간'으로 연장"
2,2022-09-30,롤러코스터 증시… “코스피 1900대까지 하락할 수도”
3,2022-09-30,"한투 ""SK하이닉스, 반도체 악재 선반영…목표주가 유지"""
4,2022-09-30,"""세계국채지수(WGBI) 편입되면 금리·환율 안정 기대"""
5,2022-09-30,"“두산, 경쟁사 대비 로봇사업 가치 현저히 저평가” 목표가 18만 원"
6,2022-09-30,"올리패스, 나스닥 상장 제약사 美 '반다'와 공동 R&D 계약"
7,2022-09-30,NH투자증권·웨이버스 등 42개사 의무보유 해제…1억7296만주
8,2022-09-30,"""실적은 좋은데"" WCP 상장 첫날 폭락..반등 가능성은?"
9,2022-09-30,"비패밀리 메신저, 10월 출석체크 오픈 이벤트 실시"


In [20]:
realtime_news=pd_news.to_spark()
realtime_news.show()

+----------+---------------------------------+
|   STD_DAY|                            TITLE|
+----------+---------------------------------+
|2022-09-30|   "상장사, 4분기 더 강한 충격...|
|2022-09-30| 미래에셋증권, 미 주식 거래 하...|
|2022-09-30|   롤러코스터 증시… “코스피 19...|
|2022-09-30|   한투 "SK하이닉스, 반도체 악...|
|2022-09-30|    "세계국채지수(WGBI) 편입되...|
|2022-09-30|  “두산, 경쟁사 대비 로봇사업 ...|
|2022-09-30| 올리패스, 나스닥 상장 제약사 ...|
|2022-09-30|   NH투자증권·웨이버스 등 42개...|
|2022-09-30|     "실적은 좋은데" WCP 상장 ...|
|2022-09-30|  비패밀리 메신저, 10월 출석체...|
|2022-09-30|  유니온커뮤니티, 55억 규모 육...|
|2022-09-30|   애드바이오텍, 6회차 CB 전환...|
|2022-09-30|  현대퓨처넷, 한화건설과 185억...|
|2022-09-30| 안다자산운용 "SK케미칼 자사주...|
|2022-09-30| 한창, 진도 자원순환센터 개소 ...|
|2022-09-30|  미래에셋 글로벌엑스 재팬, ET...|
|2022-09-30| 박민영과 열애설 난 빗썸 회장?...|
|2022-09-30|미래에셋증권, 미국주식 거래 아...|
|2022-09-30|    [특징주] 체면 구긴 '대어' ...|
|2022-09-30|     파인텍, 자사주 25만9819주...|
+----------+---------------------------------+
only showing top 20 rows



In [21]:
realtime_news.write.jdbc(url=JDBC['url'], table='REALTIME_NEWS', mode='append', properties=JDBC['props'])

In [47]:
realtime_news_s = realtime_news.to_spark()
realtime_news_s


AttributeError: 'DataFrame' object has no attribute 'to_spark'